# Recommendation System

## Import Libraries

In [12]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors  
from scipy.sparse import csr_matrix

## Load Data

In [13]:
ratings = pd.read_csv('data/ratings.csv')
movies = pd.read_csv('data/movies.csv')

## Create User Item Matrix
- Serves as the baseline of our model
- Maps the movie and user IDs with their indices
- Final result is a sparse matrix where each entry is a user's rating for the movie

In [14]:
def create_matrix(df):
    N = len(df['userId'].unique())
    M = len(df['movieId'].unique())
    user_mapper = dict(zip(np.unique(df['userId']), list(range(N))))
    movie_mapper = dict(zip(np.unique(df['movieId']), list(range(M))))
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df['userId'])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df['movieId'])))
    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]
    X = csr_matrix((df['rating'], (movie_index, user_index)), shape=(M, N))
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)


## Find Similar Movies


In [15]:
def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    neighbour_ids = []
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k+=1
    knn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric=metric)
    knn.fit(X)
    movie_vec = movie_vec.reshape(1,-1)
    neighbour = knn.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

movie_titles = dict(zip(movies['movieId'], movies['title']))

## Test System

In [ ]:
movie_id = 3
K=10
similar_ids = find_similar_movies(movie_id, X, k=K)
movie_title = movie_titles[movie_id]

print(f'Since you watched {movie_title}, your top {K} recommendations are:')
for i in similar_ids:
    print(movie_titles[i])

Since you watched Grumpier Old Men (1995), your top 10 recommendations are:
Grumpy Old Men (1993)
Striptease (1996)
Nutty Professor, The (1996)
Twister (1996)
Father of the Bride Part II (1995)
Broken Arrow (1996)
Bio-Dome (1996)
Truth About Cats & Dogs, The (1996)
Sabrina (1995)
Birdcage, The (1996)
